### Resets all variables in the workspace

In [7]:
%reset -f

### Importing necessary libraries

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import pickle

### Declaring functions

In [9]:
#Function that reads all the pickle files in a folder, returns list of file names.
def read_files_in_folder(folder_path: str) -> list[str]:

    files = [f for f in os.listdir(folder_path) if f.endswith('.p')]
    
    return files

In [10]:
#Function that reads all the pickle files in a folder, returns the unpacked transcrips as an array.
def extract_transcripts(folder_path: str, ticker: str) -> np.ndarray: 

    file_path = os.path.join(folder_path, ticker) #File path.

    with open(file_path, 'rb') as file: #Open file.

        content = pickle.load(file) #Load content.
        transcripts = np.empty(len(content), dtype=object) #Create empty array.

        for i, entry in enumerate(content): #Iterate over the content.

            transcripts[i] = entry["transcript"] #Extract the transcript.
            
    return transcripts #Return the array of transcripts.

### Main function

In [11]:
folder_path = 'tickers' #Path to the folder containing the pickle files.
files = read_files_in_folder(folder_path) #List of all the pickle files in the folder.

transcripts_list = [] #Initialize an empty list to store the transcripts.

#Loop through all files and extract transcripts.
for file in files:

    transcripts = extract_transcripts(folder_path, file) #Extract the transcripts.
    transcripts_list.append(transcripts) #Append the transcripts to the list.

df_transcripts = pd.DataFrame(transcripts_list, index=files) #Create a DataFrame from the list.
